# About this notebook:
- We will be using articles IDs to collect each articles' information, including:
    - Title
    - Abstract
    - Given Mesh Terms
    - Publication Date
- This notebook is part of a series of notebooks under 'Step 2 Articles Collection', where each notebook contain articles published in a particular year.

# Import Libraries and Articles' IDs

In [1]:
# import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from Bio import Entrez
import xml.etree.ElementTree as ET

import time
from tqdm import tqdm

# Set the email address associated with your NCBI account
Entrez.email = "geok1723@gmail.com"

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


df=pd.read_csv("../Step_1_ID_Collection/IDs_2019.csv")

In [2]:
df.head()

,Date_str,ID,Date,Month,Year
0,2019/01/25,30674258,2019-01-25,1,2019
1,2019/01/31,30697481,2019-01-31,1,2019
2,2019/01/15,30640189,2019-01-15,1,2019
3,2019/01/09,30618847,2019-01-09,1,2019
4,2019/01/31,30698037,2019-01-31,1,2019


# Collecting Articles in groups of 100<br>
> The IDs are split into mini-dataframes containing 100 IDs for scraping. <br>
> Output will contain about 360 mini dataframes of articles that are published in this particular year. <br>
> This process was executed using Kaggle, and resulting dataframes are stored in the respective zip-folder. <br>

In [3]:
# determine the group number for each row
group_num = (df.index // 100) + 1

# map the group number to a group label (G1 to G6)
group_label = 'G' + group_num.astype(str)

# add the new column to the DataFrame
df['sub_group'] = group_label

# print(df)
# print(df['sub_group'].nunique())

df['Date']=pd.to_datetime(df['Date'],yearfirst=True)

# group the DataFrame by "Sub_group"
grouped = df.groupby('sub_group')

# create an empty dictionary to store the smaller DataFrames
df_dict = {}

# loop over the groups and create smaller DataFrames
for subgroup, subgroup_df in grouped:
    # construct the name of the smaller DataFrame
    year = subgroup_df['Year'].iloc[0]
    month = subgroup_df['Date'].iloc[0].strftime('%m')
    df_name = f"df_{year}_{month}_{subgroup}"
    # add the smaller DataFrame to the dictionary
    df_dict[df_name] = subgroup_df.copy()

df_ls = list(df_dict.keys())

df_ls.sort()

In [7]:
output_df = {}
for grp in tqdm(df_ls):
    _df = df_dict[grp]
    _s = _df[['ID','Date_str']] 

    print(f"Creating Dataframe of articles for {grp}")
    abstracts = []
    titles = []
    ids = []
    MeSH = []
    Pub_Date = []

    for index,row in tqdm(_s.iterrows()):	

        pub_date = row['Date_str']
        pmid = row['ID']


        try:
            handle = Entrez.efetch(db="pubmed", id=pmid, rettype="abstract", retmode="xml")

            xml_data = handle.read()
            handle.close()

            #obtain abstract and title
            root = ET.fromstring(xml_data)
            title = root.find(".//ArticleTitle").text.strip()
            abst = root.findall(".//AbstractText")

            abstract_text = ""
            for abstract in abst:
                abstract_text += abstract.text

            
            #obtain MeSH
            mesh_terms = ['']
            mesh_list = root.findall(".//MeshHeadingList/MeshHeading")
            for mesh in mesh_list:
                descriptor_name = mesh.find(".//DescriptorName").text
                qualifiers = mesh.findall(".//QualifierName")
                qualifier_names = [qualifier.text for qualifier in qualifiers]
                mesh_terms.append((descriptor_name, qualifier_names))
                
            MeSH.append(mesh_terms)            
            ids.append(pmid)
            abstracts.append(abstract_text)
            titles.append(title)
            Pub_Date.append(pub_date)

        except:
            print(f"Error here in ID{pmid}")

    # Store the data in a pandas dataframe
    data = {"ID": ids, "title": titles,"Pub_Date":Pub_Date, "abstract": abstracts, "MeSH_term":MeSH}
    data_df = pd.DataFrame(data)
    df_name = f"ABSTRACT{grp}"
    output_df[df_name] = data_df
    data_df.to_csv(f"{df_name}.csv", index=False)
    print(output_df.keys())

  0%|          | 0/360 [00:00<?, ?it/s]

Creating Dataframe of articles for df_2019_01_G1



0it [00:00, ?it/s]
1it [00:00,  1.73it/s]
2it [00:01,  1.59it/s]
3it [00:01,  1.67it/s]
4it [00:02,  1.63it/s]
5it [00:02,  1.70it/s]
6it [00:03,  1.63it/s]
7it [00:04,  1.66it/s]
8it [00:04,  1.63it/s]
9it [00:05,  1.68it/s]
10it [00:05,  1.71it/s]
11it [00:06,  1.65it/s]
12it [00:07,  1.61it/s]

Error here in ID30615545



13it [00:07,  1.67it/s]
14it [00:08,  1.62it/s]
15it [00:09,  1.57it/s]
16it [00:09,  1.58it/s]
17it [00:10,  1.65it/s]
18it [00:11,  1.59it/s]
19it [00:11,  1.65it/s]
20it [00:12,  1.71it/s]

...

Error here in ID31841335



65it [00:40,  1.60it/s]
66it [00:41,  1.50it/s]
67it [00:42,  1.59it/s]
68it [00:42,  1.62it/s]
69it [00:43,  1.55it/s]
70it [00:44,  1.40it/s]
71it [00:44,  1.50it/s]
72it [00:45,  1.51it/s]
73it [00:46,  1.50it/s]
74it [00:46,  1.56it/s]
75it [00:47,  1.50it/s]
76it [00:48,  1.44it/s]
77it [00:49,  1.44it/s]
78it [00:49,  1.47it/s]
79it [00:50,  1.50it/s]
80it [00:51,  1.43it/s]
81it [00:51,  1.44it/s]
82it [00:52,  1.53it/s]
83it [00:53,  1.50it/s]
84it [00:53,  1.50it/s]
85it [00:54,  1.51it/s]
86it [00:54,  1.58it/s]
87it [00:55,  1.55it/s]
88it [00:56,  1.56it/s]
89it [00:56,  1.51it/s]
90it [00:58,  1.19it/s]
91it [00:58,  1.33it/s]
92it [00:59,  1.36it/s]
93it [01:00,  1.42it/s]

Error here in ID35117043



94it [01:00,  1.42it/s]
95it [01:01,  1.41it/s]
96it [01:02,  1.45it/s]
97it [01:02,  1.53it/s]
98it [01:03,  1.57it/s]
99it [01:03,  1.61it/s]
100it [01:04,  1.55it/s]
100%|██████████| 360/360 [6:39:31<00:00, 66.59s/it]

dict_keys(['ABSTRACTdf_2019_01_G1', 'ABSTRACTdf_2019_01_G10', 'ABSTRACTdf_2019_01_G11', 'ABSTRACTdf_2019_01_G12', 'ABSTRACTdf_2019_01_G13', 'ABSTRACTdf_2019_01_G14', 'ABSTRACTdf_2019_01_G15', 'ABSTRACTdf_2019_01_G16', 'ABSTRACTdf_2019_01_G17', 'ABSTRACTdf_2019_01_G18', 'ABSTRACTdf_2019_01_G19', 'ABSTRACTdf_2019_01_G2', 'ABSTRACTdf_2019_01_G20', 'ABSTRACTdf_2019_01_G21', 'ABSTRACTdf_2019_01_G22', 'ABSTRACTdf_2019_01_G23', 'ABSTRACTdf_2019_01_G24', 'ABSTRACTdf_2019_01_G25', 'ABSTRACTdf_2019_01_G26', 'ABSTRACTdf_2019_01_G27', 'ABSTRACTdf_2019_01_G28', 'ABSTRACTdf_2019_01_G29', 'ABSTRACTdf_2019_01_G3', 'ABSTRACTdf_2019_01_G30', 'ABSTRACTdf_2019_01_G4', 'ABSTRACTdf_2019_01_G5', 'ABSTRACTdf_2019_01_G6', 'ABSTRACTdf_2019_01_G7', 'ABSTRACTdf_2019_01_G8', 'ABSTRACTdf_2019_01_G9', 'ABSTRACTdf_2019_02_G31', 'ABSTRACTdf_2019_02_G32', 'ABSTRACTdf_2019_02_G33', 'ABSTRACTdf_2019_02_G34', 'ABSTRACTdf_2019_02_G35', 'ABSTRACTdf_2019_02_G36', 'ABSTRACTdf_2019_02_G37', 'ABSTRACTdf_2019_02_G38', 'ABSTRACTd